# Integral Analysis of PHOTON<sub>256</sub> permutation function

# Importing necessary libraries

In [1]:
import random
import copy
from collections import deque
from IPython.display import display, Math , HTML
import galois
GF = galois.GF(2**4)
print(GF.properties)

Galois Field:
  name: GF(2^4)
  characteristic: 2
  degree: 4
  order: 16
  irreducible_poly: x^4 + x + 1
  is_primitive_poly: True
  primitive_element: x


# Utility functions

In [2]:
#------------------------------------------- Utility Functions -----------------------------------------------
def xor_all_ele(lst):
    a = 0
    for i in lst:
        a ^= i
    return a

# Matrix Set

In [3]:
class PHOTON_Matrix_Set_8x8:
    def __init__(self):
        a = range(16)
        self.set = []
        self.n = 8
        self._constant_matrix = [ [random.choice(a) for j in range(self.n)] for i in range(self.n)]
        for i in a:
            temp = copy.deepcopy(self._constant_matrix)
            temp[0][0] = i
            self.set.append(temp)
    def isAll(self):
        temp = [[set() for j in range(self.n)] for i in range(self.n)]
        for m in self.set:
            for row in range(self.n):
                for col in range(self.n):
                    temp[row][col].add(m[row][col])
        temp2 = [[0 for j in range(self.n)] for i in range(self.n)]
        for row in range(self.n):
            for col in range(self.n):
                if len(temp[row][col])== 16:
                    temp2[row][col] =  True
                else:
                    temp2[row][col] =  False
        return temp2
    def isConstant(self):
        temp = [[set() for j in range(self.n)] for i in range(self.n)]
        for m in self.set:
            for row in range(self.n):
                for col in range(self.n):
                    temp[row][col].add(m[row][col])
        temp2 = [[0 for j in range(self.n)] for i in range(self.n)]
        for row in range(self.n):
            for col in range(self.n):
                if len(temp[row][col])== 1:
                    temp2[row][col] =  True
                else:
                    temp2[row][col] =  False
        return temp2
    def isBalanced(self):
        temp = [[0 for j in range(self.n)] for i in range(self.n)]
        for m in self.set:
            for row in range(self.n):
                for col in range(self.n):
                    temp[row][col] ^= m[row][col]
        temp2 = [[0 for j in range(self.n)] for i in range(self.n)]
        for row in range(self.n):
            for col in range(self.n):
                if temp[row][col]== 0:
                    temp2[row][col] =  True
                else:
                    temp2[row][col] =  False
        return temp2
    def is_a_b_c(self):
        temp = [[set() for j in range(self.n)] for i in range(self.n)]
        for m in self.set:
            for row in range(self.n):
                for col in range(self.n):
                    temp[row][col].add(m[row][col])
        temp2 = [["*" for j in range(self.n)] for i in range(self.n)]
        for row in range(self.n):
            for col in range(self.n):
                l = len(temp[row][col])
                if l == 16:
                    temp2[row][col] =  'A'
                elif l == 1:
                    temp2[row][col] =  'C'
                elif xor_all_ele(temp[row][col]) == 0:
                    temp2[row][col] =  'B'
        return temp2
    def copy(self):
        temp = PHOTON_Matrix_Set_8x8()
        temp.set = copy.deepcopy(self.set)
        temp._constant_matrix = copy.deepcopy(self._constant_matrix)
        return temp
    def property_summary(self):
        print("Property")
        print_matrix(self.is_a_b_c())
    def get_property_summary_matrix_str(self):
        array = self.is_a_b_c()
        matrix = ''
        for row in array:
            try:
                matrix += " & ".join(map(str,row))
            except TypeError:
                pass
            matrix = matrix + r' \\ '
        matrix = r' \begin{bmatrix} '+matrix+r' \end{bmatrix} '
        return matrix
    def get_latex_table_code(self):
        array = self.is_a_b_c()
        matrix = ''
        for row in array:
            try:
                matrix += " & ".join([ " {"+i+"} " for i in map(str,row)])
            except TypeError:
                matrix += f'{row}&'
            matrix = matrix[:-1] + r' \\ \hline '
        matrix = r'\begin{table}[] \begin{tabular}{|c|c|c|c|c|c|c|c|} \hline '+matrix+r' \end{tabular} \end{table} '
        return matrix

# PHOTON<sub>256</sub> - Permutation

In [4]:
# ---------------------------------------- PHOTON-256-PERMUTATION ---------------------------------------------
## SBOX LIST
sbox_list = [0xc,5,6,0xb,9,0,0xa,0xd,3,0xe,0xf,8,4,7,1,2]
def list_64_to_8x8_matrix(s) -> list[list[int]]:
    assert len(s) == 64
    m = [[s[i+(j*8)] for j in range(8)] for i in range(8)]
    return m
def matrix_8x8_to_hex_list(m : list[list[int]]):
    lst = []
    for col in range(8):
        for row in range(8):
            lst.append(m[row][col])
    return lst
## ----------------------------------------------- Round Functions -------------------------------------------
## 1.ADD-CONSTANT
def add_constant(X : list,k):
    new_X = copy.deepcopy(X)
    RC = [1, 3, 7, 14, 13, 11, 6, 12, 9, 2, 5, 10]
    IC = [0, 1, 3, 7, 15, 14, 12, 8]
    for i in range(8):
        new_X[i][0] = new_X[i][0] ^ RC[k] ^ IC[i]
    return new_X
## 2.SUB-CELL
def sub_cell(X):
    new_X = copy.deepcopy(X)
    for i in range(8):
        for j in range(8):
            new_X[i][j] = sbox_list[new_X[i][j]]
    return new_X
## 3.SHIFT-ROW
def shift_row(X):
    new_X = copy.deepcopy(X)
    for i in range(8):
        temp = deque(new_X[i])
        temp.rotate(-1*i)
        new_X[i] = list(temp)
    return new_X
## 4.MIX-COLUMN-SERIAL
def serial(lst):
    M = []
    for i in range(7):
        a = [0 for j in range(8)]
        a[i+1] = 1
        M.append(a)
    M.append(copy.deepcopy(lst))
    return M
def matrix_mul(m1,m2):
    new_m = [[0 for j in range(8)] for i in range(8)]
    for i in range(8):
        for j in range(8):
            s = 0
            for temp in range(8):
                s ^= int(GF(m1[i][temp]) * GF(m2[temp][j]))
            new_m[i][j] = s
    return new_m
def mix_column_serial(X):
    new_X = copy.deepcopy(X)
    M = serial([2, 4, 2, 11, 2, 8, 5, 6])
    M8 = matrix_mul(M,M)
    for i in range(6):
        M8 = matrix_mul(M8,M)
    new_X = matrix_mul(M8,new_X)
    return new_X
## PHOTON 256 PERMUTATION FUNCTION
def PHOTON_256_single_round(X,i):
    abcd = []
    X = copy.deepcopy(X)
    #1
    X = add_constant(X,i)
    abcd.append(X)
    X = copy.deepcopy(X)
    #2
    X = sub_cell(X)
    abcd.append(X)
    X = copy.deepcopy(X)
    #3
    X = shift_row(X)
    abcd.append(X)
    X = copy.deepcopy(X)
    #4
    X = mix_column_serial(X)
    abcd.append(X)
    X = copy.deepcopy(X)
    
    return X
def perform_PHOTON_on_whole_set(S:PHOTON_Matrix_Set_8x8,r=1):
    S = S.copy()
    for idx,m in enumerate(S.set):
        S.set[idx] = PHOTON_256_single_round(m,r)
    return S
#------------------------------ Function to perform operations on each function ------------------------------
# 1. Add-Constant
# 2. Sub-Cell
# 3. Shift-Row
# 4. Mix-Column-Serial
def add_constant_on_set(A,k) -> PHOTON_Matrix_Set_8x8:
    A = A.copy()
    for idx,m in enumerate(A.set):
        A.set[idx] = add_constant(m,k)
    return A
def sub_cell_on_set(A) -> PHOTON_Matrix_Set_8x8:
    A = A.copy()
    for idx,m in enumerate(A.set):
        A.set[idx] = sub_cell(m)
    return A
def shift_row_on_set(A) -> PHOTON_Matrix_Set_8x8:
    A = A.copy()
    for idx,m in enumerate(A.set):
        A.set[idx] = shift_row(m)
    return A
def mix_column_serial_on_set(A) -> PHOTON_Matrix_Set_8x8:
    A = A.copy()
    for idx,m in enumerate(A.set):
        A.set[idx] = mix_column_serial(m)
    return A

# Main Visualization function

In [5]:
def visualize_one_round_PHOTON256_properties(A:PHOTON_Matrix_Set_8x8,k=1):
    print(f"- - - Round {k} - - -")
    print("After Add-constant:")
    A = add_constant_on_set(A,k)
    display(Math(A.get_property_summary_matrix_str()))
    print("After Sub-Cell:")
    A = sub_cell_on_set(A)
    display(Math(A.get_property_summary_matrix_str()))
    print("After Shift-Row:")
    A = shift_row_on_set(A)
    display(Math(A.get_property_summary_matrix_str()))
    print("After Mix-Column-serial:")
    A = mix_column_serial_on_set(A)
    display(Math(A.get_property_summary_matrix_str()))
    return A

# Test

In [6]:
a = PHOTON_Matrix_Set_8x8()

In [8]:
a = visualize_one_round_PHOTON256_properties(a,1)

- - - Round 1 - - -
After Add-constant:


<IPython.core.display.Math object>

After Sub-Cell:


<IPython.core.display.Math object>

After Shift-Row:


<IPython.core.display.Math object>

After Mix-Column-serial:


<IPython.core.display.Math object>

In [9]:
a = visualize_one_round_PHOTON256_properties(a,2)

- - - Round 2 - - -
After Add-constant:


<IPython.core.display.Math object>

After Sub-Cell:


<IPython.core.display.Math object>

After Shift-Row:


<IPython.core.display.Math object>

After Mix-Column-serial:


<IPython.core.display.Math object>

In [10]:
a = visualize_one_round_PHOTON256_properties(a,3)

- - - Round 3 - - -
After Add-constant:


<IPython.core.display.Math object>

After Sub-Cell:


<IPython.core.display.Math object>

After Shift-Row:


<IPython.core.display.Math object>

After Mix-Column-serial:


<IPython.core.display.Math object>

In [11]:
a = visualize_one_round_PHOTON256_properties(a,4)

- - - Round 4 - - -
After Add-constant:


<IPython.core.display.Math object>

After Sub-Cell:


<IPython.core.display.Math object>

After Shift-Row:


<IPython.core.display.Math object>

After Mix-Column-serial:


<IPython.core.display.Math object>

# - - - END - - -